![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from ast import literal_eval
import json

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

import re
from datetime import date, datetime, timedelta
import numpy as np

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [3]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [4]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [5]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [6]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [7]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

## Функции

In [24]:
# Убирает "лишние" кухни для последующей категоризации признака

def cuisine_shorter(cuisine_list, top_cuisines):
    cuisine_short_list = []
    for cuisine in cuisine_list:
        if cuisine in top_cuisines:
            cuisine_short_list.append(cuisine)
        else:
            cuisine_short_list.append('national_cuisine')
    return list(set(cuisine_short_list))


# разделяет отзыв на состовляющие - дату и время

def text_date_separator(row):
    text = row[1:-1].split('], [')[0] + ']'
    date = '[' + row[1:-1].split('], [')[1]
    return[text,date]


# возвращает разницу в днях между датами в отзывах
# если одна дата или нет дат возвращает отрицательные числа

def date_diff(row):
    if len(row) == 2:
        delta = row[1]-row[0]
        delta = delta.days
        return delta
    elif len(row) == 1:
        return -1
    else:
        return -2
    
    
# Разбивает текст на список из отдельных слов

def text_spliter(row):
    row = row.replace("['",'')
    row = row.replace("']",'')
    row = row.lower()
    row = re.sub('[^a-z]', ' ', row)
    row = re.sub(" +", " ", row)
    row = row.split(' ')
    return(row)


# функция обработки отзыва 
# работает как социальный рейтинг, хорошие слова увеличивают рейтинг плохие уменьшают

def review_rating(review,df_bad_lexicon,df_good_lexicon):
    row = text_spliter(review)
    
    bad_lexicon = df_bad_lexicon[0].unique()
    good_lexicon = df_good_lexicon[0].unique()
    
    review_rating = 0
    for word in row:
        if word in good_lexicon:
            review_rating += 1
        elif word in bad_lexicon:
            review_rating -= 1
    return (review_rating)

## Предобработка

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [143]:
data.info()

In [144]:
# Столбец Number of Reviews
data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')
data['Number of Reviews'].fillna(0, inplace=True)

# Cuisine Style для начала заполним пропуски отсутствующей кухней
data['Cuisine_Style_isNAN'] = pd.isna(data['Cuisine Style']).astype('uint8')
data['Cuisine Style'] = data['Cuisine Style'].fillna("['No_Cuisine']")

# Reviews для начала заполним просто соответствующей пустотой и выделим пустые отзывы
data.Reviews.fillna('[[], []]', inplace = True)
data['Reviews_isNAN'] = data.Reviews.isin(['[[], []]']).astype('uint8')

# Price range заполним наиболее встречающимся диапазоном цен
data['Price_Range_isNAN'] = pd.isna(data['Price Range']).astype('uint8')
data['Price Range'].fillna('$$ - $$$', inplace = True)

### 2. Обработка признаков

In [145]:
# Уберем ненужные признаки из tripadviser
data.drop(columns = ['URL_TA','ID_TA'], inplace = True)

In [146]:
data.sample(5)

In [147]:
# Преобразуем Price Range (категорийный признак)
data['Price Range'] = data['Price Range'].apply(lambda x: 1 if x == '$' else (2 if x == '$$ - $$$' else 3))

# Преобразуем Cuisine Style в список посчитаем число представленных кухонь перед обработкой 
data['Cuisine Style'] = data['Cuisine Style'].apply(literal_eval)
data['Cuisine_count'] = data['Cuisine Style'].apply(lambda x: len(x))

# Разобъем Reviews на отзывы и даты преобразуем их в нужные типы
data['review_txt'] = data.Reviews.apply(lambda x: text_date_separator(x)[0])
data['review_date'] = data.Reviews.apply(lambda x: text_date_separator(x)[1])
data['review_date'] = data['review_date'].apply(literal_eval)
data['review_date'] = data['review_date'].apply(lambda x: sorted([pd.to_datetime(i).date() for i in x]))
data.drop(columns = ['Reviews'], inplace = True)

In [148]:
data

## Подключим сторонние источники

In [149]:
# Подсоединяем dataset с городами
DATA_DIR = '/kaggle/input/world-cities/'
df_cities = pd.read_csv(DATA_DIR+'/worldcities.csv')
df_cities.sample(5)

In [150]:
df_cities[df_cities.city == 'Porto']

In [151]:
# получим только города европы для этого ограничим широту и долготу на европу, чтобы не попали лишние города
# Порту иногда называют по-другому
data.City = data.City.str.replace('Oporto', 'Porto')

df_cities = df_cities[(df_cities.city_ascii.isin(data.City.unique())) 
          & (df_cities.lng >= -20) 
          & (df_cities.lng <= 60)
          & (df_cities.lat >= 30)
          & (df_cities.lat <= 70)]

# Проверим о каких городах нет информации
# display(set(data.City.unique()) - set(df_cities[(df_cities.city_ascii.isin(data.City.unique()))].city_ascii.unique()))



# убедимся, что не попало лишних городов
# display(df_cities[df_cities.city_ascii.isin(data.City.unique())].city_ascii.value_counts())

# Оставим только то, что можно использовать из df городов
df_cities = df_cities[['city_ascii','capital','population']]
df_cities.fillna('admin', inplace = True)

# числовой признак столицы
df_cities['capital'] = df_cities['capital'].isin(['primary']).astype('uint8')

df_cities.columns = ['City','capital','population']
data = data.merge(df_cities, how = 'left', on = ['City'])

In [152]:
data.info()

In [153]:
data[data.capital.isna()].City.value_counts()

In [154]:
df_cities

In [155]:
# подтянем dataset с лексикой
DATA_DIR = '/kaggle/input/opinion-lexicon/opinion_lexicon/'
df_good_lexicon = pd.read_table(DATA_DIR+"/positive-words.txt", comment=';', header=None)
df_bad_lexicon = pd.read_table(DATA_DIR+"/negative-words.txt", comment=';', header=None)

In [156]:
# Применим функцию рейтинга на основе отзывов
data['review_rating'] = data.review_txt.apply(lambda x: review_rating(x, df_bad_lexicon, df_good_lexicon))

In [157]:
data.sample(5)

## Создадим новые признаки

In [158]:
# Разница между отзывами в днях
data['days_diff'] = data['review_date'].apply(lambda x: date_diff(x))


plt.rcParams['figure.figsize'] = (10,7)
data['days_diff'].hist(bins=100)
# у многих ресторанов отзывы отсутствуют

In [159]:
# Сетевой ресторан или нет
net = data.groupby(['Restaurant_id']).agg(
            cities = ('City', 'nunique')
            ).reset_index()
net['network'] = net.cities.apply(lambda x: 0 if x == 1 else 1)
data = data.merge(net[['Restaurant_id','network']], how='left', on = ['Restaurant_id'])

data['network'].value_counts()
# сетевых ресторанив гораздо больше

In [160]:
# кухни

# создадим series с названиями кухонь и числом их вхождений в список кухонь
cuisine_list = data['Cuisine Style'].tolist()
cuisine_dict = {}
for cuisines in cuisine_list:
    for cuisine in cuisines:
        if cuisine in cuisine_dict.keys():
            cuisine_dict[cuisine] = cuisine_dict[cuisine]+1
        else:
            cuisine_dict[cuisine] = 1

cuisine_series = pd.Series(cuisine_dict)

# Рассмотри состав кухонь
plt.rcParams['figure.figsize'] = (10,25)
cuisine_series.sort_values(ascending=True).plot(kind='barh')

In [161]:
print(cuisine_series.drop('No_Cuisine').sum())
print(len(cuisine_series))
print(cuisine_series.drop('No_Cuisine').head(30).sum())
print(round(cuisine_series.drop('No_Cuisine').head(30).sum()/cuisine_series.sum()*100,2))

# по  графику видно, что есть много разных национальных кухонь, 
# из 126 кухонь первые 30 не включая отсутствующие кухни составляет почти 70 процентов всех кухонь поэтому заменим осталььные кухни и отсутствующие одной национальной
top_cuisines = cuisine_series.drop('No_Cuisine').head(30).keys()
data['Cuisine Style'] = data['Cuisine Style'].apply(lambda x: cuisine_shorter(x, top_cuisines))

In [162]:
# посмотрим со сколькими ресторанами конкурирует конкретный ресторан
restorants_in_city = data.groupby('City')['Ranking'].count().to_dict()
data['Restaurants_Count'] = data['City'].map(restorants_in_city)

In [163]:
# приведем параметры к взвешенному значению
data['Weighed_Rank'] = data.Ranking / data.Restaurants_Count

In [164]:
# посмотрим на топ 10 городов
for x in (data['City'].value_counts())[5:5].index:
    data['Weighed_Rank'][data['City'] == x].hist(bins=100)
plt.show()

In [165]:
# число отзывов на 1000 человек
data['Number of Reviews Weight'] = data['Number of Reviews']/data['population']*1000

In [166]:
data.describe()

In [167]:
data[['Ranking', 'Price Range',
       'Number of Reviews', 'Number_of_Reviews_isNAN',
       'Cuisine_Style_isNAN', 'Reviews_isNAN', 'Price_Range_isNAN',
       'Cuisine_count', 'capital', 'population',
       'review_rating', 'days_diff', 'network', 'Restaurants_Count',
       'Weighed_Rank','Number of Reviews Weight']].hist(figsize=(20, 20), bins=100);
plt.tight_layout()

### Посмотрим распределение признака

In [168]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [169]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [170]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [171]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


### Посмотрим распределение целевой переменной

In [172]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [173]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [174]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [175]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

Вообще благодаря визуализации в этом датасете можно узнать много интересных фактов, например:
* где больше Пицерий в Мадриде или Лондоне?
* в каком городе кухня ресторанов более разнообразна?

придумайте свои вопрос и найдите на него ответ в данных)

## добавим dummies переменные

In [176]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
df_1 = pd.get_dummies(data, columns=[ 'City',], dummy_na=True)

In [177]:
cuisines_df = pd.get_dummies(data['Cuisine Style'].apply(pd.Series).stack()).sum(level=0)

In [178]:
df2 = pd.concat([df_1,cuisines_df], axis=1)

In [179]:
df2.drop(columns=['Restaurant_id','Cuisine Style', 'review_txt', 'review_date'], inplace = True)

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [25]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    data = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    data.drop(columns = ['URL_TA','ID_TA'], inplace = True)
    
    # ################### 2. NAN ############################################################## 
    data['Number of Reviews'].fillna(0, inplace=True)

    # Столбец Number of Reviews
    data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')
    data['Number of Reviews'].fillna(0, inplace=True)

    # Cuisine Style для начала заполним пропуски отсутствующей кухней
    data['Cuisine_Style_isNAN'] = pd.isna(data['Cuisine Style']).astype('uint8')
    data['Cuisine Style'] = data['Cuisine Style'].fillna("['No_Cuisine']")

    # Reviews для начала заполним просто соответствующей пустотой и выделим пустые отзывы
    data.Reviews.fillna('[[], []]', inplace = True)
    data['Reviews_isNAN'] = data.Reviews.isin(['[[], []]']).astype('uint8')

    # Price range заполним наиболее встречающимся диапазоном цен
    data['Price_Range_isNAN'] = pd.isna(data['Price Range']).astype('uint8')
    data['Price Range'].fillna('$$ - $$$', inplace = True)
    
    
    # ################### 3. Encoding ############################################################## 

    # Преобразуем Price Range (категорийный признак)
    data['Price Range'] = data['Price Range'].apply(lambda x: 1 if x == '$' else (2 if x == '$$ - $$$' else 3))

    # Преобразуем Cuisine Style в список посчитаем число представленных кухонь перед обработкой 
    data['Cuisine Style'] = data['Cuisine Style'].apply(literal_eval)
    data['Cuisine_count'] = data['Cuisine Style'].apply(lambda x: len(x))

    # Разобъем Reviews на отзывы и даты преобразуем их в нужные типы
    data['review_txt'] = data.Reviews.apply(lambda x: text_date_separator(x)[0])
    data['review_date'] = data.Reviews.apply(lambda x: text_date_separator(x)[1])
    data['review_date'] = data['review_date'].apply(literal_eval)
    data['review_date'] = data['review_date'].apply(lambda x: sorted([pd.to_datetime(i).date() for i in x]))
    data.drop(columns = ['Reviews'], inplace = True)
    
    # ################### 4. Feature Engineering ####################################################
    
    # Подсоединяем dataset с городами
    DATA_DIR = '/kaggle/input/world-cities/'
    df_cities = pd.read_csv(DATA_DIR+'/worldcities.csv')

    # Порту иногда называют по-другому
    data.City = data.City.str.replace('Oporto', 'Porto')

    # получим только города европы для этого ограничим широту и долготу на европу, чтобы не попали лишние города
    df_cities = df_cities[(df_cities.city_ascii.isin(data.City.unique())) 
              & (df_cities.lng >= -20) 
              & (df_cities.lng <= 60)
              & (df_cities.lat >= 30)
              & (df_cities.lat <= 70)]

    # Оставим только то, что можно использовать из df городов
    df_cities = df_cities[['city_ascii','capital','population']]
    df_cities.fillna('admin', inplace = True)

    # числовой признак столицы
    df_cities['capital'] = df_cities['capital'].isin(['primary']).astype('uint8')
    
    # джоин
    df_cities.columns = ['City','capital','population']
    data = data.merge(df_cities, how = 'left', on = ['City'])
    
    
    # подтянем dataset с лексикой
    DATA_DIR = '/kaggle/input/opinion-lexicon/opinion_lexicon/'
    df_good_lexicon = pd.read_table(DATA_DIR+"/positive-words.txt", comment=';', header=None)
    df_bad_lexicon = pd.read_table(DATA_DIR+"/negative-words.txt", comment=';', header=None)
    # Применим функцию рейтинга на основе отзывов
    data['review_rating'] = data.review_txt.apply(lambda x: review_rating(x,df_bad_lexicon,df_good_lexicon))
    
    
    # Разница между отзывами в днях
    data['days_diff'] = data['review_date'].apply(lambda x: date_diff(x))
    
    # Сетевой ресторан или нет
    net = data.groupby(['Restaurant_id']).agg(
                cities = ('City', 'nunique')
                ).reset_index()
    net['network'] = net.cities.apply(lambda x: 0 if x == 1 else 1)
    data = data.merge(net[['Restaurant_id','network']], how='left', on = ['Restaurant_id'])
    
    
    # кухни
    # создадим series с названиями кухонь и числом их вхождений в список кухонь
    cuisine_list = data['Cuisine Style'].tolist()
    cuisine_dict = {}
    for cuisines in cuisine_list:
        for cuisine in cuisines:
            if cuisine in cuisine_dict.keys():
                cuisine_dict[cuisine] = cuisine_dict[cuisine]+1
            else:
                cuisine_dict[cuisine] = 1

    cuisine_series = pd.Series(cuisine_dict)
    
    top_cuisines = cuisine_series.drop('No_Cuisine').head(30).keys()
    data['Cuisine Style'] = data['Cuisine Style'].apply(lambda x: cuisine_shorter(x, top_cuisines))
    
    
    # посмотрим со сколькими ресторанами конкурирует конкретный ресторан
    restorants_in_city = data.groupby('City')['Ranking'].count().to_dict()
    data['Restaurants_Count'] = data['City'].map(restorants_in_city)
    
    
    # приведем параметры к взвешенному значению
    data['Weighed_Rank'] = data.Ranking / data.Restaurants_Count
    
    # число отзывов на 1000 человек
    data['Number of Reviews Weight'] = data['Number of Reviews']/data['population']*1000
    
    
    
    # Dummies
    df_1 = pd.get_dummies(data, columns=[ 'City',], dummy_na=True)
    cuisines_df = pd.get_dummies(data['Cuisine Style'].apply(pd.Series).stack()).sum(level=0)
    df_output = pd.concat([df_1,cuisines_df], axis=1)
    
    
    # ################### 5. Clean #################################################### 
    
    df_output.drop(columns=['Restaurant_id','Cuisine Style', 'review_txt', 'review_date'], inplace = True)
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [26]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [27]:
df_preproc.info()

In [28]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [29]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [30]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [31]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [32]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [33]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [34]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [35]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [36]:
test_data.sample(10)

In [37]:
test_data = test_data.drop(['Rating'], axis=1)

In [38]:
sample_submission

In [39]:
predict_submission = model.predict(test_data)

In [40]:
predict_submission

In [41]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
